In [1]:
import csv
import json
import os
import requests

In [4]:
import requests
import json
import os
import csv

def fetch_manifest_data(manifest_url):
    """
    Fetch and return JSON data from a given IIIF manifest URL.
    """
    response = requests.get(manifest_url)
    response.raise_for_status()
    return response.json()

def create_collection_manifest(manifest_urls, output_path):
    """
    Create a IIIF 3.0 Collection manifest from a list of manifest URLs.
    """
    collection_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": "http://example.org/iiif/collection.json",
        "type": "Collection",
        "label": {"en": ["Collection of IIIF Manifests"]},
        "items": []
    }

    for url in manifest_urls:
        manifest_data = fetch_manifest_data(url)
        manifest_id = manifest_data.get("id", manifest_data.get("@id"))
        label = manifest_data.get("label", {"en": ["No label"]})

        if manifest_id:
            collection_manifest["items"].append({
                "id": manifest_id,
                "type": "Manifest",
                "label": label
            })
        else:
            print(f"Warning: 'id' or '@id' not found in manifest data from {url}")

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(collection_manifest, f, indent=2)

def create_book_manifest(manifest_urls, output_path):
    """
    Create a IIIF 3.0 Book manifest with canvases for pages from a list of manifest URLs.
    """
    book_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": "http://example.org/iiif/book.json",
        "type": "Manifest",
        "label": {"en": ["Book of IIIF Manifests"]},
        "items": []
    }

    for url in manifest_urls:
        manifest_data = fetch_manifest_data(url)
        manifest_id = manifest_data.get("id", manifest_data.get("@id"))
        label = manifest_data.get("label", {"en": ["No label"]})
        
        # Define default dimensions for the Canvas
        width = manifest_data.get("width", 1000)
        height = manifest_data.get("height", 1000)

        # Create annotation for painting
        annotation_page = {
            "id": f"{manifest_id}/annotation-page",
            "type": "AnnotationPage",
            "items": [
                {
                    "id": f"{manifest_id}/annotation",
                    "type": "Annotation",
                    "motivation": "painting",
                    "body": {
                        "id": f"{manifest_id}/image",
                        "type": "Image",
                        "format": "image/jpeg",
                        "width": width,
                        "height": height
                    },
                    "target": manifest_id
                }
            ]
        }

        if manifest_id:
            book_manifest["items"].append({
                "id": manifest_id,
                "type": "Canvas",
                "label": label,
                "width": width,
                "height": height,
                "items": [annotation_page]
            })
        else:
            print(f"Warning: 'id' or '@id' not found in manifest data from {url}")

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(book_manifest, f, indent=2)

def main(input_csv, manifest_type):
    """
    Main function to generate either a collection or book manifest based on user input.
    """
    output_json = './output/collection.json' if manifest_type == 'collection' else './output/book.json'

    manifest_urls = []
    with open(input_csv, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            manifest_urls.append(row['manifest_json'])

    if manifest_type == 'collection':
        create_collection_manifest(manifest_urls, output_json)
    elif manifest_type == 'book':
        create_book_manifest(manifest_urls, output_json)
    else:
        print("Invalid manifest type. Use 'collection' or 'book'.")

# Example usage:
# main("input.csv", "collection") or main("input.csv", "book")


In [5]:
main("/Users/maples/GitHub/Paper2Pixels/data/collection.csv", "collection")  # For a collection manifest
main("/Users/maples/GitHub/Paper2Pixels/data/collection.csv", "book")        # For a book manifest
